In [1]:
import sys
sys.path.append('../src/collaborative_filtration')
sys.path.append('../src/database inference')


sys.path.insert(1, '../src/data')

from get_users_info import *
from get_brand_category_info import *
from get_preference_matrix import *
from get_popular_items import *
import implicit
import faiss
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.spatial.distance import euclidean, cosine
from sklearn.metrics import precision_score
from itertools import islice
from pymongo import MongoClient
import certifi
from get_train_test_matrices import *
from validation_pipeline import *
from validation_pipeline import *
from collaborative_inference import *
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 1000)

In [2]:
Path(os.path.abspath('')).parent / 'data/raw/goods.xlsx'

goods = pd.read_excel(Path(os.path.abspath('')).parent / 'data/raw/goods.xlsx')

goods.loc[:,'brand_categ'] = list(zip(goods.brand, goods.categ))

brand_categ = goods.loc[:,['id','brand_categ']].set_index('id')

In [86]:
# находим трейн-тест для отдельных товаров
product_user = get_pref_matrix(to_csv=False, grouped=False)

user_product_cut = cut_user_item(product_user.T)

user_product_old_cut = get_old_user_item(user_product_cut, days_back = 20, grouped = False)

user_product_diff = get_binarized_differences(user_product_cut, user_product_old_cut, do_first='binarize')

In [337]:
user_product_diff

4294967295 2000000135847 200000013583 2000000135755  \
1586861088574635654           0             0            0             0   
1589124488521527290           0             0            0             0   
1589209329103247949           0             0            0             0   
15945421401032860445          0             0            0             0   
1594795295110685242           0             0            0             0   
...                         ...           ...          ...           ...   
1646096524939107924           0             0            0             0   
1646144915655748076           0             0            0             0   
1646150483359159998           0             0            0             0   
1646161262880979442           0             0            0             0   
1645991948113671749           0             0            0             0   

                     2000000135748 2000000135731 2000000135724 2000000135717  \
1586861088574635654              0             0             0             0   
1589124488521527290              0             0             0             0   
1589209329103247949              0             0             0             0   
15945421401032860445             0             0             0             0   
1594795295110685242              0             0             0             0   
...                            ...           ...           ...           ...   
1646096524939107924              0             0             0             0   
1646144915655748076              0             0             0             0   
1646150483359159998              0             0             0             0   
1646161262880979442              0             0             0             0   
1645991948113671749              0             0             0             0   

                     20000001357 2000000135694 2000000135687 200000013567  \
1586861088574635654            0             0             0            0   
1589124488521527290            0             0             0            0   
1589209329103247949            0             0             0            0   
15945421401032860445           0             0             0            0   
1594795295110685242            0             0             0            0   
...                          ...           ...           ...          ...   
1646096524939107924            0             0             0            0   
1646144915655748076            0             0             0            0   
1646150483359159998            0             0             0            0   
1646161262880979442            0             0             0            0   
1645991948113671749            0             0             0            0   

                     2000000135663 2000000135656 2000000135649 2000000135632  \
1586861088574635654              0             0             0             0   
1589124488521527290              0             0             0             0   
1589209329103247949              0             0             0             0   
15945421401032860445             0             0             0             0   
1594795295110685242              0             0             0             0   
...                            ...           ...           ...           ...   
1646096524939107924              0             0             0             0   
1646144915655748076              0             0             0             0   
1646150483359159998              0             0             0             0   
1646161262880979442              0             0             0             0   
1645991948113671749              0             0             0             0   

                     2000000135618 2000000135601 2000000135595 2000000135588  \
1586861088574635654              0             0             0             0   
1589124488521527290              0             0             0             0   
1589209329103247949  

In [83]:
# находим трейн-тест по категориям
pref_matrix = get_pref_matrix(to_csv=False)
item_user = pref_matrix.drop(columns=['id_s','id_list','item_total'], index = ['user_total'])

user_item_cut = cut_user_item(item_user)

user_item_old_cut = get_old_user_itemm(user_item_cut, days_back=20)

user_item_diff = get_binarized_differences(user_item_cut, user_item_old_cut, do_first='binarize')

user_item_diff.shape

(108, 575)

In [100]:
def als_fit_model(user_item_cut):
    user_item_array = np.array(user_item_cut).astype('float32')

    if str(user_item_array.flags)[17:22] == 'False':
        user_item_array = user_item_array.copy(order='C')

    als_model = implicit.als.AlternatingLeastSquares(factors = 50)
    als_model.fit(csr_matrix(user_item_array.T))
    return als_model

In [102]:
als_model_prods = als_fit_model(user_product_cut)

  0%|          | 0/15 [00:00<?, ?it/s]

In [207]:
user_ids = [user_product_cut.index.get_loc(i) for i in user_product_diff.index]

In [328]:
user_ids2 = [user_item_cut.index.get_loc(i) for i in user_item_diff.index]

In [331]:
def get_als_recommends(user_item_cut, user_ids, n = 20, products = False, model = als_model):
    user_dict = {}
    for user in tqdm(user_ids):
        user_str = user_item_cut.index[user]
        user_dict[user_str] = {}
        rec_items, rec_scores  =  np.array(model.recommend(user, csr_matrix(user_item_cut), N=n))[:,0].astype(int), \
        np.array(model.recommend(user, csr_matrix(user_item_cut), N=n))[:,1]

        if products:
            recs_df = pd.DataFrame({'item_num':rec_items, 
                                    'score':rec_scores, 
                                    'id':user_item_cut.columns[rec_items]}).set_index('id').join(brand_categ)
        else:
            recs_df = pd.DataFrame({'item_num':rec_items, 
                                    'score':rec_scores, 
                                    'id':user_item_cut.columns[rec_items]}).set_index('id')
        recommends = recs_df['score'].reindex(index = user_item_cut.iloc[user].index, 
                                              fill_value = 0).sort_values(ascending = False)
        preferred_binary = (user_item_cut.iloc[user] > 0).astype(int)

        user_dict[user_str]['recommends'] = recommends
        user_dict[user_str]['preferred_binary'] = preferred_binary
    return user_dict


In [343]:
user_cat_dict = get_als_recommends(user_item_old_cut, user_ids2)

 95%|███████████████████████████████████████  | 103/108 [00:05<00:00, 17.90it/s]


IndexError: index 2863 is out of bounds for axis 0 with size 2863

In [ ]:
for user in user_cat_dict:
    print(precision_at_k(user_item_diff, user_item_cut, user_cat_dict, user, k = 30), recall_at_k(user_item_diff, user_cat_dict, user, k = 30))
    

In [297]:
for user in user_dict:
    print(precision_at_k(user_product_diff, user_product_cut, user_dict, user, k = 30), recall_at_k(user_product_diff, user_dict, user, k = 30))
    

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0


### раздел с категориями

In [341]:
user_item_array = np.array(user_item_old_cut).astype('float32')

if str(user_item_array.flags)[17:22] == 'False':
    user_item_array = user_item_array.copy(order='C')

In [342]:
als_model = implicit.als.AlternatingLeastSquares(factors = 50)
als_model.fit(csr_matrix(user_item_array.T))

  0%|          | 0/15 [00:00<?, ?it/s]

In [96]:
items = user_item_cut.columns

In [99]:
pd.DataFrame({'items': items[np.array(als_model.similar_items(23))[:,0].astype(int)], 
              'scores': np.array(als_model.similar_items(23))[:,1]})

,items,scores
0,"(Alanui, Джемперы и свитеры)",1.000000
1,"(Thom Browne, Пальто и плащи)",0.632996
2,"(Rifatto, Куртки и пуховики)",0.619450
3,"(Thom Browne, Куртки и пуховики)",0.618929
4,"(Thom Browne, Штаны и брюки)",0.609131
5,"(Thom Browne, Рубашки)",0.601162
6,"(Thom Browne, Пиджаки и жилеты)",0.581478
7,"(Comme Des Garcons Junya Watanabe, Рубашки)",0.568926
8,"(Thom Browne, Джемперы и свитеры)",0.568231
9,"(Doublet, Галстуки, запонки, носки)",0.532135


In [30]:
ids, scores= als_model.similar_items(0)

# display the results using pandas for nicer formatting
pd.DataFrame({"artist": items[ids], "score": scores})

ValueError: too many values to unpack (expected 2)

In [ ]:
# написать пайплайн оценки
# два разных для категорий и отдельных вещей
# оценить метрики такие же

In [166]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from implicit.datasets.lastfm import get_lastfm

artists, users, artist_user_plays = get_lastfm()

0.00B [00:00, ?B/s]

In [174]:
user_plays = artist_user_plays.T.tocsr()

In [177]:
model = implicit.als.AlternatingLeastSquares(factors=64, regularization=0.05)
model.fit(2 * user_plays)

  0%|          | 0/15 [00:00<?, ?it/s]

In [194]:
user_plays[userids].T

<292385x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 49271 stored elements in Compressed Sparse Column format>

In [340]:
userids = np.arange(1000)
model.recommend([1,2], user_plays.T)

ValueError: shapes (575,50) and (2,50) not aligned: 50 (dim 1) != 2 (dim 0)

In [ ]:
# написать сравнивалку векторов прошлого и настоящего
# лайтфм: собрать фичи юзера, фичи айтема и фичи взаимодействия
# для новой матрицы: мелтим юзер-айтем, лефтджоин юзер фичи и айтем фичи, таргет бинарный - взаимодействовал с айтемом или нет
# отдельными функциями создать матрицы юзер фичи и айтем фичи
